In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from glob import glob
import os
import json
import pandas as pd
import numpy as np
from common import *

In [ ]:
# load benchmark reference paths
exp_name = "t2b"
responses = {'P1': None, 'P2': None} # note participant initials have been replaced with P1, P2
benchmark_dir = os.path.join(exp_name, 'benchmark')
answers_paths = [os.path.join(benchmark_dir, f'{exp_name}-part-{i}', 'answers.json') for i in range(5)]    

In [ ]:
# load human responses
data = []
for r in list(responses.keys()):
    r_path = os.path.join(exp_name, 'benchmark', f'{r}.xlsx')    
    responses[r] = pd.read_excel(r_path, header=0, index_col=0)

# iterate over partitions
for pi, part in enumerate([os.path.join(benchmark_dir,  f'{exp_name}-part-{i}') for i in range(5)]):
    col_name = os.path.basename(part)
    answers_path = os.path.join(part, 'answers.json')
    
    # load answers
    with open(answers_path, 'r') as f:
        answers = json.load(f)
    
    # append responses
    for r in list(responses.keys()):
        for i, a in enumerate(answers):            
            answers[i][f'{r}'] = responses[r].iloc[i, pi].lower()    
    
    # append to master list
    data += answers

In [ ]:
# load machine responses
machine_response_path = os.path.join("t1-final", f"t1-final-benchmark-results.json")
with open(machine_response_path, 'r') as f:
    m_result = json.load(f)
    m_result = pd.DataFrame.from_dict(m_result)
    m_result['image_path'] = m_result['image_path'].apply(lambda x: os.path.basename(x))
    m_result.columns = ['image_path', 'M', 'IoU']
    m_result = m_result.set_index('image_path')

In [ ]:
m_result

In [ ]:
benchmark = pd.DataFrame.from_dict(data).dropna().drop(columns=['test_idx', 'label_path']).drop_duplicates("image_path")
benchmark['image_path'] = benchmark['image_path'].apply(lambda x: os.path.basename(x))
benchmark = benchmark.set_index('image_path')

# override labels
benchmark[benchmark != 'rock'] = 'artefact'

# join with machine results
benchmark = pd.concat((benchmark, m_result), axis=1)

# rename columns
benchmark.index.names = ["image"]
benchmark.columns = ["label"] + benchmark.columns.values.tolist()[1:]
benchmark

In [ ]:
print(benchmark.label.value_counts())
print('Total', len(benchmark))

In [ ]:
# accuracy
cols = benchmark.columns[1:].tolist()
cols.remove('IoU')

for p in cols:
    print(p)
    result = benchmark[p] == benchmark.label
    score = sum(result) / len(benchmark)
    print(f"{sum(result)}, {score:.3f}")

In [ ]:
# IoU
print(benchmark.IoU.mean(), benchmark.IoU.std())

In [ ]:
# time 6.79 hrs